# Introduction
~~This document analyzes the resulting patterns using a script in "research \ src \ SPY_patterns_scanner \ main.py". We will try to evaluate the quality of patterns consisting of a different number of candles, with a different level of similarity of patterns, and also evaluate the forecast, which is expressed in the price change after the pattern has appeared.~~

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pickle import load
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

os.chdir('../..')

In [2]:
def weights_matrix(pic):
    weights = np.zeros((height, width))
    d_value = 2 * height / ((height - pic) * (height - pic + 1) + (pic - 1) * pic)
    for c in range(width):
        for j in range(height):
            try:
                weights[j, c] = 1 - abs(pic[c] - j) * d_value[c]
            except Exception as ex:
                print(ex)
    return weights

## 6 prices patterns
In this part, we will look at the patterns that were obtained with the similarity 80 and 90%. This time we did not calculate the patterns for 70% accuracy as they are quite different.

In [3]:
similarity = 90   # 80
height, width = 10, 6

patterns = load(open(f'src/SPY_patterns_scanner/output/{similarity}/{width}x{height}/patterns.pkl', 'rb'))
patterns.head()

,date,index,prototype,prototype_label,diff,min_price,max_price,all_prices,code,similarity,price_fall,price_rise
0,2020-01-02 09:30:00.002,0,0,1,0.03,323.54,323.57,"[323.54, 323.56, 323.54, 323.56, 323.54, 323.57]","[0, 2, 0, 2, 0, 3]",100.000000,-0.01,0.03
1,2020-01-02 09:30:00.410,29,0,0,0.03,323.56,323.59,"[323.56, 323.58, 323.57, 323.58, 323.57, 323.59]","[0, 2, 1, 2, 1, 3]",93.939394,-0.03,0.02
2,2020-01-02 09:30:01.472,60,0,0,0.02,323.58,323.60,"[323.58, 323.59, 323.58, 323.6, 323.58, 323.6]","[0, 1, 0, 2, 0, 2]",90.119151,-0.05,0.03
3,2020-01-02 09:33:47.367,1420,0,0,0.02,323.72,323.74,"[323.73, 323.74, 323.72, 323.74, 323.72, 323.74]","[1, 2, 0, 2, 0, 2]",91.593353,-0.03,0.09
4,2020-01-02 09:37:52.832,2674,0,0,0.03,323.79,323.82,"[323.79, 323.81, 323.8, 323.81, 323.8, 323.82]","[0, 2, 1, 2, 1, 3]",93.939394,-0.24,0.05


Below is the sorted number of first 20 patterns belonging to each of the prototypes.

### Top 20 most frequent patterns

In [4]:
count_of_patterns = patterns[['prototype', 'code']].groupby(['prototype']).agg('count').sort_values(by=[
    'code'], ascending=False)
count_of_patterns.head(20).T

prototype,5,6,9,7,8,34,20,22,18,79,50,81,42,49,43,104,21,19,127,60
code,16219,9484,3728,2622,2579,2537,2053,1531,1205,1055,767,655,611,550,481,385,367,352,341,322


### Total amount of patterns template

In [5]:
count_of_patterns.count()

code    291
dtype: int64

## Patterns visualization

In [7]:
prototype_num = 126

for i in range(1, count_of_patterns.loc[prototype_num].item()):
    if i > 20:
        print('THERE ARE MORE THAN 100 PATTERNS!')
        break
    
    patterns_df = patterns[patterns.prototype == prototype_num]
    date = patterns_df.date.iloc[i]   #.replace(microsecond=0)
    
    prototype_code = patterns_df.iloc[0].code
    candidate_code = patterns_df.iloc[i].code

    prototype_weights = weights_matrix(prototype_code)
    candidate_weights = weights_matrix(candidate_code)

    fig = go.Figure()
    # Add traces
    fig.add_trace(go.Scatter(x=list(range(width)), y=prototype_code,
                        #mode='lines+markers',
                        line=dict(color='orange', width=4, dash='dashdot'),
                        name='prototype'))
    fig.add_trace(go.Scatter(x=list(range(width)), y=candidate_code,
                        #mode='lines',
                        line = dict(color='royalblue', width=4, dash='dash'),
                        name='candidate'))
    fig.update_layout(title=f"{i}. Candidate {candidate_code}. Date {date}. Similarity {similarity}")
    print(f'\nPROTOTYPE CODE: {prototype_code}')
    fig.show()


PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]


...


PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]



PROTOTYPE CODE: [1 2 1 0 1 0]


### Summary
~~It can be noted that these patterns have a very strange form of persistent price fluctuations without an obvious trend. Let's try to introduce a target variable for the price change and set the price threshold at 5 cents.~~

## General summary
~~In this study, we tried to find patterns that will give a stable forecast for price changes. We tested large patterns consisting of 15 prices and small ones of 5. The dominant set of patterns does not have a trend and they do not have a probability advantage of at least 1%. By introducing a threshold on the target variable, the study came to two conclusions that flattening the distributions of price change leads to the fact that the forecast ceases to show any probabilistic advantage. In the case of searching for a small group of stable patterns, we come to the conclusion that their number is so insignificant that it is 0.4% of the entire sample.~~